<a href="https://colab.research.google.com/github/kridtapon/GAPO-VI-Trend-Tracker/blob/main/GAPO_VI_Trend_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Gopalakrishnan Range Index (GAPO)
def calculate_gapo(df, period=14):
    """
    Calculate Gopalakrishnan Range Index (GAPO).
    """
    range_high = df['High'].rolling(window=period).max() - df['Low'].rolling(window=period).min()
    gapo = (df['High'] - df['Low']) / range_high
    return gapo.fillna(0)

# Function to calculate Vortex Indicator (VI)
def calculate_vortex(df, period=14):
    """
    Calculate Vortex Indicator (VI+ and VI-).
    """
    df['TR'] = np.maximum(df['High'] - df['Low'],
                          np.abs(df['High'] - df['Close'].shift(1)),
                          np.abs(df['Low'] - df['Close'].shift(1)))

    df['VM+'] = np.where(df['High'] - df['High'].shift(1) > 0, df['High'] - df['High'].shift(1), 0)
    df['VM-'] = np.where(df['Low'].shift(1) - df['Low'] > 0, df['Low'].shift(1) - df['Low'], 0)

    df['VI+'] = df['VM+'].rolling(window=period).sum() / df['TR'].rolling(window=period).sum()
    df['VI-'] = df['VM-'].rolling(window=period).sum() / df['TR'].rolling(window=period).sum()

    return df['VI+'], df['VI-']

# Define the stock symbol and time period
symbol = 'META'  # Example stock (Meta Platforms)
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Gopalakrishnan Range Index (GAPO)
df['GAPO'] = calculate_gapo(df, period=14)

# Calculate Vortex Indicator (VI+ and VI-)
df['VI+'], df['VI-'] = calculate_vortex(df, period=14)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit signals based on GAPO and Vortex Indicator
df['Entry'] = (df['GAPO'] < df['GAPO'].shift(1)) & (df['VI+'] > df['VI-'])
df['Exit'] = (df['GAPO'] > df['GAPO'].shift(1)) & (df['VI-'] > df['VI+'])

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           149866.205857
Total Return [%]                        49.866206
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                      11722.259749
Max Drawdown [%]                        49.516022
Max Drawdown Duration                       748.0
Total Trades                                   51
Total Closed Trades                            51
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            41.176471
Best Trade [%]                            46.9465
Worst Trade [%]                        -12.596954
Avg Winning Trade [%]                    9.515885
Avg Losing Trade [%]                    -4.410659
